## Guide: Finding NFB topologies

In this notebook we present our metodology to add selective pressure to the genetic algorithm to support the discovery of NFB-like topologies. For more details on the single steps, check also `synthetic_evolution_sensitivity.ipynb`.

In [1]:
using SynthEvo

# load the CRN
C = make_FullyConnectedNonExplosive_CRN(3)
t0 = 10.
t1 = 20.
input = 1.

# prepare ODE problems for the state and sensitivities
base_problem_ode = make_base_problem_for_FCNE_CRN(C, C.ext_ode, t0, input) # this is faster to compute
base_problem_ext = make_base_problem_for_FCNE_CRN(C, C.ext_ode, t0, input)

nothing

ArgumentError: ArgumentError: Package SynthEvo not found in current path.
- Run `import Pkg; Pkg.add("SynthEvo")` to install the SynthEvo package.

#### Near-Perfect Adaptation setup

We will omit some details as they are already coveres in the guide `symbolic_gradient_descent.ipynb`.

In [2]:
# set up the loss function for near-perfect adaptation
loss1 = SynthEvo.adaptation_loss(C, 1, 3, 10., 20.)
loss2 = SynthEvo.sensitivity_loss(C, 1, 3, 0.25, 10., 10.5)
loss3 = SynthEvo.steady_state_loss(C, 2, 10., 20., 6, 16.)
loss4 = SynthEvo.regularization_loss(C, 1)
l = SynthEvo.weighted_loss([loss1, loss2, loss3, loss4], [10,1,100,0.02])

perturbation_list = [0, 0.25, 0.75, 1, 1.5, 2, 3, 4, 5, 6, 7, 8]

gd_options = (
    n_iter = 50,
    verbose = false,
    use_random_perturbation = false, 
    use_pruning_heuristic = false,
    clip_value = nothing,
    use_gradient_normalization = false,
    use_gradient_noise = false,
    fraction_gradient_noise = 0.01,
    alpha = 0.1,
    use_adam = false,
    ADAM_beta1 = 0.9,
    ADAM_beta2 = 0.9,
    use_adagrad = true
)

gd_perturbation_options = (
    t0 = t0,
    t1 = t1,
    loss_fun = l,
    input = input,
    perturbation_list = perturbation_list
)

# for the NFB run, we use functions for the choices of the GA actions 
# this helps in tuning the probabilities based on the rank of the individual particle in 
# the population (genetic pool) 

ga_options = (
    genetic_pool_size = 100,
    elite = 0,
    worst = 0,
    death_rate = (rank) -> 0.05*(rank),
    mutation_rate = (rank) -> 0.25,
    gradient_mutation_rate = (rank) -> 0.01*(1-rank),
    duplication_rate = (rank) -> 0.20*(1-rank),
    crossover_rate = (rank) -> 0.0*(1-rank),
    max_generations = 10,
    p_cross = 0.05,
    dp = 0.05,
)

ga_perturbation_options = (
    use_random_perturbation = false,
    t0 = t0,
    t1 = t1,
    loss_fun = l,
    input = input,
    perturbation_list = perturbation_list
)

mutate_with_GD = (p) -> SynthEvo.symbolic_gradient_descent(p, C, gd_options, gd_perturbation_options).parameters

ga_loss = SynthEvo.prepare_GA_loss(C, base_problem_ode, ga_perturbation_options)

nothing

UndefVarError: UndefVarError: `SynthEvo` not defined

### running the genetic algorithm

This can be simply done by initializing a random state and then calling the `symbolic_evolve_NFB` function iteratively. This function is analog to `symbolic_evolve` but it is designed to 
have random perturbations in the degradation of the output species. This is something that IFF topologies cannot comensate for. In addition we remove elitism, to avoid selectig for topologies that are not robust to parameter changes.

We need to find the parameter related to the outupt species degradation: 

In [3]:
using Catalyst
output_degradation_parameter = 29 
reactions(C.crn)[29+1] # as the 1st is the input reaction

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


Then we can proceed with the evolution.

In [ ]:
using ProgressBars
max_generations = ga_options.max_generations

state = initialize_state(C.number_of_parameters, ga_options, "LHC")

iter = ProgressBar(1:max_generations)
for i in iter
    state = SynthEvo.symbolic_evolve_NFB(ga_loss, state, mutate_with_GD, ga_options, output_degradation_parameter, 0.1)
    set_postfix(iter, avg_loss=state.history.mean_loss[end], best_loss=state.history.best_loss[end])
end

### Plotting the results

In [ ]:
plot_history(state)

In [ ]:
argmin(state.fitness)

In [ ]:
opt_index = argmin(state.fitness)

SynthEvo.quick_trajectory_plot(C, state.pool[opt_index], 1, gd_perturbation_options.perturbation_list, t0, t1, 3)

### Evaluating the homeostatic indicator

In [ ]:
homeostatic_coefs = [
    compute_homeostatic_coefficient(C, p, 1, 2.5, 10., 20.)
    for p in state.pool
]

nothing

In [ ]:
using LaTeXStrings, Plots
# we remove the outliers (networks that are not adapting)
x = [h.A_22_A_31 for h in homeostatic_coefs if abs(h.coefficient)<1]
y = [h.A_21_A_32 for h in homeostatic_coefs if abs(h.coefficient)<1]
scatter(x, y, title="Homeostatic indicator components", xlabel=L"A_{22}A_{31}", ylabel=L"A_{21}_A_{32}", legend=false, label=false)
plot!([-1000, 1000], [-1000, 1000], color=:red, linestyle=:dash, label="theoretical IFF line", legend=:bottomright)
xlims!(minimum(x)-0.1, maximum(x)+0.1)
ylims!(minimum(y)-0.1, maximum(y)+0.1)

In [ ]:
NFB_indexes = [i for i in 1:length(homeostatic_coefs) if abs(homeostatic_coefs[i].A_22_A_31) + abs(homeostatic_coefs[i].A_21_A_32) < 0.3]
for i in NFB_indexes
    println("Network $i :: hc=", homeostatic_coefs[i].coefficient, " :: f=", state.fitness[i])
end

In [ ]:
idx = 3
println("Network $idx :: hc=", homeostatic_coefs[idx].coefficient, " :: f=", state.fitness[idx])